<a href="https://colab.research.google.com/github/marianasmoura/tecnicas-de-otimizacao/blob/main/Gradiente_Conjugado_(Fletcher_Reeves).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UNIVERSIDADE FEDERAL DO PIAUÍ

CURSO DE GRADUAÇÃO EM ENGENHARIA ELÉTRICA

DISCIPLINA: TÉCNICAS DE OTIMIZAÇÃO

DOCENTE: ALDIR SILVA SOUSA

DISCENTE: MARIANA DE SOUSA MOURA

Atividade 4: Otimização Irrestrita pelo Método do Gradiente Conjugado (Fletcher-Reeves)

# **Método do Gradiente Conjudado (Fletcher-Reeves) - Multivariável**

#**Método da Bisseção Monovariável**

Trecho responsável por realizar a minimização do valor de lamda, que corresponde a taxa com a qual a otimização da função é feita.

In [ ]:
import numpy as np
import sympy as sym   #Para criar variáveis simbólicas.

class Params:
  def __init__(self,f,vars,eps,a,b):
    self.f = f
    self.a = a
    self.b = b    
    self.vars = vars #variáveis simbólicas
    self.eps = eps

def eval(sym_f,vars,x):
   map = dict()
   map[vars[0]] = x
   return sym_f.subs(map)    

import pandas as pd
import math

def bissecao(params):
   n = math.ceil( -math.log(params.eps/(params.b-params.a),2) )
   f = params.f
   diff = sym.diff(f) #retorna a derivada simbólica de f
   
   a = params.a
   b = params.b

   for k in range(n):
           x = (b + a)/2
           
           fx  = eval(f,params.vars,x)
           dfx = eval(diff,params.vars,x)  

           if (dfx == 0): break  # Mínimo encontrado. Parar.
           if (dfx > 0 ):
               #Passo 2
               b = x
           else:
               #Passo 3
               a = x  
   x = (a+b)/2        
   return x


#**Cômputo do gradiente e avaliação das funções nos pontos**

Nesta seção, têm-se definições de funções para calcular os valores do gradiente e da hessiana utilizando-se a função em termos de variáveis simbólicas. Também realiza-se aqui, por meio dessas definições de funções, a substituição das variáveis simbólicas por valores numéricos passados como parâmetro.

In [ ]:
# Função para o cálculo do gradiente e da Hessiana 
import sympy as sym   #Para criar variáveis simbólicas.
def gradiente_simbolico(funcao,variaveis):
     g = [sym.diff(funcao,x) for x in variaveis]
     return g
   
# Função para substituição dos valores nas variáveis simbólicas
def eval_simbolica(f,variaveis,x):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   return float(f.subs(mp))

# Função para substituição de f(x + lambda*d)
def eval_simb(f,variaveis,x):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   return f.subs(mp)

def eval_gradiente(grad_simb,variaveis,x):
   g = [ eval_simbolica(f,variaveis,x) for f in grad_simb]
   return g 

In [ ]:
import numpy as np
import sympy as sym 

class Parametros:
  def __init__(self,f,d1f,variaveis,x,eps):
    self.f = f
    self.d1f = d1f
    self.x = x
    self.eps = eps
    self.variaveis = variaveis #variáveis simbólicas

In [ ]:
import pandas as pd
import math
lmbd = sym.Symbol('lmbd')

def gradiente_conjugado(p):
      f = p.f
      d1f = p.d1f
      eps = p.eps
      x = p.x
      variaveis = p.variaveis
      n = 0
      cols = ['x','y','df(x)','Tolerância']    
      table = pd.DataFrame([], columns=cols)

      y = x
      d = eval_gradiente(d1f,variaveis,x)
      d = - np.array(d)
      k = j = 1
      
      tolerancia = np.linalg.norm(d)
      while (tolerancia > eps): 
          v = 0 
          v = x + d*lmbd 
          subP = eval_simb(f,variaveis,v)
          arg = Params(subP,[lmbd],eps,0,1)
          lmb = bissecao(arg)
         
          G1 = eval_gradiente(d1f,variaveis,y)
          y = y + d*lmb
          G2 = eval_gradiente(d1f,variaveis,y)

          a = pow(np.linalg.norm(G2),2)/pow(np.linalg.norm(G1),2)
          d = d*a - eval_gradiente(d1f,variaveis,y)
          print('d ',d)
          print('y ',y)
         
          tolerancia =  np.linalg.norm(d) # calcula o valor da tolerância - critério de parada
          print('tolerância ',tolerancia)
          print('=====================================================')
          x = y
          sP = eval_simbolica(f,variaveis,x)
          row = pd.DataFrame([[x,y,d,tolerancia]],columns=cols)
          table = table.append(row, ignore_index=True) # concatena valores de cada iteração
      return y,sP,table

**1.** Considere o seguinte problema:

Minimizar $\sum_{i=2}^{n} [100(x_i-x^2_{i-1})^2 + (1-x_{i-1})^2]$

Resolva para n = 5, 10, e 50. Iniciando do ponto $x_0 = [-1.2,1.0,-1.2,1.0,...]$

In [ ]:
# Para n = 5
import numpy as np
import sympy as sym 
import pandas as pd

variaveis = list(sym.symbols("x:5"))
print('variaveis ',variaveis)
c = variaveis
print('c ',c)
f1 = 0
for i in range(1,5):
    f1 = f1 + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 

x = []    
for i in range(1,6):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  
print('x ',x)

eps = 1e-5
grad = gradiente_simbolico(f1,c)
d1f = eval_gradiente(grad,c,x)
print(grad)
print(f1)
print(d1f)
p  = Parametros(f1,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)


variaveis  [x0, x1, x2, x3, x4]
c  [x0, x1, x2, x3, x4]
x  [-1.2, 1, -1.2, 1, -1.2]
[-400*x0*(-x0**2 + x1) + 2*x0 - 2, -200*x0**2 - 400*x1*(-x1**2 + x2) + 202*x1 - 2, -200*x1**2 - 400*x2*(-x2**2 + x3) + 202*x2 - 2, -200*x2**2 - 400*x3*(-x3**2 + x4) + 202*x3 - 2, -200*x3**2 + 200*x4]
(1 - x0)**2 + (1 - x1)**2 + (1 - x2)**2 + (1 - x3)**2 + 100*(-x0**2 + x1)**2 + 100*(-x1**2 + x2)**2 + 100*(-x2**2 + x3)**2 + 100*(-x3**2 + x4)**2
[-215.5999999999999, 792.0, -655.5999999999999, 792.0, -440.0]
d  [346.31407322 116.64653708 147.69085615 -23.0650806  162.25959417]
y  [-0.91461029 -0.04837036 -0.33218231 -0.04837036 -0.61757202]
tolerância  426.86369575921236
d  [ 41.38495491  -3.71055021 -41.48628338  45.66385677  -5.72110716]
y  [ 0.56632414  0.45044238  0.2993845  -0.147003    0.07629473]
tolerância  74.60236770190072
d  [ -6.81182371   5.15390173  -0.47058492  13.07426706 -12.20090523]
y  [ 0.69025294  0.439331    0.17515227 -0.01026088  0.05916266]
tolerância  19.823815047497135
d  [ 0.496

Nota-se que o método apresenta uma grande variação do valor do gradiente, entrando e saindo da região factível até a convergência. Para valores de tolerância de ordem superior a $10^{-3}$, notou-se que a derivada se distanciava muito da solução e o método não convergia.

In [ ]:
m

array([0.99999996, 0.99999992, 0.99999984, 0.99999968, 0.99999935])

In [ ]:
fx

1.5137417942400827e-13

In [ ]:
table

,x,y,df(x),Tolerância
0,"[-0.9146102905273439, -0.048370361328125, -0.3...","[-0.9146102905273439, -0.048370361328125, -0.3...","[346.3140732236458, 116.6465370786413, 147.690...",426.863696
1,"[0.5663241427753711, 0.45044238306105383, 0.29...","[0.5663241427753711, 0.45044238306105383, 0.29...","[41.38495491380895, -3.710550209700809, -41.48...",74.602368
2,"[0.690252936138332, 0.43933100185600954, 0.175...","[0.690252936138332, 0.43933100185600954, 0.175...","[-6.811823707773481, 5.15390172512722, -0.4705...",19.823815
3,"[0.6678278420686279, 0.45629807792406674, 0.17...","[0.6678278420686279, 0.45629807792406674, 0.17...","[0.49660892718390404, -5.07995285216618, 8.555...",15.147807
4,"[0.6690800488743094, 0.443488901153735, 0.1951...","[0.6690800488743094, 0.443488901153735, 0.1951...","[-0.41363978031284687, 1.2416602270714023, 3.6...",3.915585
...,...,...,...,...
314,"[0.9999999516397206, 0.9999999067092938, 0.999...","[0.9999999516397206, 0.9999999067092938, 0.999...","[2.028457777135125e-06, 3.678512815477564e-06,...",0.000016
315,"[0.9999999527462478, 0.9999999087159289, 0.999...","[0.9999999527462478, 0.9999999087159289, 0.999...","[4.551230146932591e-06, 1.1778157042935305e-05...",0.000026
316,"[0.9999999549164434, 0.9999999143321918, 0.999...","[0.9999999549164434, 0.9999999143321918, 0.999...","[6.3392078432466145e-06, 1.5818415264572373e-0...",0.000027
317,"[0.9999999570202912, 0.9999999195819863, 0.999...","[0.9999999570202912, 0.9999999195819863, 0.999...","[5.55113852053239e-06, 9.277915122519281e-06, ...",0.000015


Apesar de patinar sobre a região de busca, o código consegue chegar a convergência com uma boa precisão na solução para n = 5. Isso só foi possível quando a tolerância utilizada foi suficentemente pequena, $10^{-5}$.

In [ ]:
# Para n = 10
import numpy as np
import sympy as sym 
import pandas as pd

variaveis = list(sym.symbols("x:10"))
c = variaveis

f1 = 0
for i in range(1,10):
    f1 = f1 + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 

x = []    
for i in range(1,11):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  
print('x ',x)

eps = 1e-8
grad = gradiente_simbolico(f1,c)
d1f = eval_gradiente(grad,c,x)
p  = Parametros(f1,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)


A saída de streaming foi truncada nas últimas 5000 linhas.
 -143.85042796   22.33489275  -97.7288496  -411.51357843 -406.50138345]
y  [0.9915688  0.99897552 1.00677059 1.00544947 0.99654015 0.99884727
 1.00524607 1.00063539 1.00648278 1.00275285]
tolerância  623.6674472318085
d  [ -42.58632721  -19.42382177  -68.10544179  118.4739772    41.58168983
 -140.40887061   16.42039087  -92.91927256 -412.68730486 -401.04569733]
y  [0.99147633 0.99893818 1.00665102 1.0056818  0.99660912 0.99857772
 1.00528792 1.00045227 1.00571168 1.00199114]
tolerância  618.3535282105588
d  [ -35.86133145  -18.98572846  -72.19691443  112.8301728    46.24384578
 -136.77167371   10.36072853  -88.32736701 -413.67242021 -396.02596154]
y  [0.99139653 0.99890179 1.0065234  1.0059038  0.99668703 0.99831462
 1.00531869 1.00027815 1.00493838 1.00123966]
tolerância  613.4370556420771
d  [ -29.17419423  -18.60766953  -76.08921692  107.06443477   50.79176116
 -132.94756012    4.16826647  -83.96191752 -414.47304373 -391.444

KeyboardInterrupt: ignored

Nota-se que para n = 10 leva muitas mais iterações porém vai chegando próximo à solução. A última vez, levou mais de 1h40min e não convergiu, porém os valores já estavam próximos à solução.

In [ ]:
# Para n = 50
import numpy as np
import sympy as sym 
import pandas as pd

variaveis = list(sym.symbols("x:50"))
c = variaveis

f1 = 0
for i in range(1,50):
    f1 = f1 + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 

x = []    
for i in range(1,51):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  
print('x ',x)

eps = 1e-2
grad = gradiente_simbolico(f1,c)
d1f = eval_gradiente(grad,c,x)
p  = Parametros(f1,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)


**2.** Resolva:

Minimizar $(x_1 - x^3_2)^2 + 3(x_1 - x_2)^4$

In [ ]:
import numpy as np
import sympy as sym 
import pandas as pd
import math

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
c = [x1,x2]

fo = (c[0] - c[1]**3)**2 + 3*(c[0] - c[1])**4

x = [1.2,1.5]

eps = 1e-4
nmax = 7
grad = gradiente_simbolico(fo,c)
p  = Parametros(fo,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)

d  [-0.0537669   0.00887579]
y  [1.26547137 1.08416432]
tolerância  0.054494576716870684
d  [-0.19024057 -0.06808596]
y  [1.25701615 1.0855601 ]
tolerância  0.2020573462052337
d  [-0.11567501  0.03133912]
y  [1.0983643  1.02877955]
tolerância  0.11984510506663787
d  [-0.00124141 -0.00330921]
y  [1.09271963 1.03030883]
tolerância  0.003534400780305511
d  [-0.00288    -0.00073402]
y  [1.09263102 1.03007261]
tolerância  0.002972066233628992
d  [-0.00536519 -0.00403691]
y  [1.08975111 1.02933861]
tolerância  0.0067143113233883635
d  [-0.01842625 -0.00743984]
y  [1.08895586 1.02874024]
tolerância  0.01987153256297003
d  [-0.22592521 -0.07212091]
y  [1.08163608 1.02578479]
tolerância  0.2371573853379651
d  [-0.00035244  0.00043289]
y  [0.98444163 0.99475792]
tolerância  0.0005582133527941423
d  [6.86994715e-06 5.66234084e-06]
y  [0.9844239 0.9947797]
tolerância  8.902711811763705e-06


In [ ]:
m

array([0.9844239, 0.9947797])

In [ ]:
fx

3.4513077624950094e-08

In [ ]:
table

,x,y,df(x),Tolerância
0,"[1.2654713745117188, 1.0841643218994141]","[1.2654713745117188, 1.0841643218994141]","[-0.05376689760702274, 0.008875788036385113]",0.054495
1,"[1.257016149189179, 1.0855601024094084]","[1.257016149189179, 1.0855601024094084]","[-0.19024056587678667, -0.06808596221235072]",0.202057
2,"[1.0983642954381123, 1.0287795515861384]","[1.0983642954381123, 1.0287795515861384]","[-0.11567501374089448, 0.03133911939537141]",0.119845
3,"[1.0927196315290642, 1.030308825631341]","[1.0927196315290642, 1.030308825631341]","[-0.001241406916267245, -0.0033092140674287076]",0.003534
4,"[1.0926310191396247, 1.0300726118952657]","[1.0926310191396247, 1.0300726118952657]","[-0.0028799980646511745, -0.0007340223734213483]",0.002972
5,"[1.0897511089655394, 1.0293386119224295]","[1.0897511089655394, 1.0293386119224295]","[-0.005365193393005559, -0.004036914217943042]",0.006714
6,"[1.0889558592002249, 1.0287402448461187]","[1.0889558592002249, 1.0287402448461187]","[-0.018426249020461284, -0.007439835578635304]",0.019872
7,"[1.081636081261402, 1.025784790142533]","[1.081636081261402, 1.025784790142533]","[-0.22592520709967429, -0.07212091386906683]",0.237157
8,"[0.9844416341030736, 0.9947579187188198]","[0.9844416341030736, 0.9947579187188198]","[-0.0003524361678068034, 0.00043288669979491435]",0.000558
9,"[0.9844238982864014, 0.9947797030867386]","[0.9844238982864014, 0.9947797030867386]","[6.869947150399018e-06, 5.662340836972056e-06]",0.000009


**3.** Resolva:



$2(x_1 - 2)^4 + (2x_1 - x_2)^2 = 4$


In [ ]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
   
variaveis = [x1,x2]
c = variaveis
x = [1.2,2]

fo = (2*(x1 - 2)**4 + (2*x1 - x2)**2 - 4)**2


eps = 1e-6
nmax = 7
grad = gradiente_simbolico(fo,c)
p  = Parametros(fo,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)

d  [-1.67627043e-03  5.76860123e-05]
y  [0.81532313 1.87670613]
tolerância  0.0016772627195273328
d  [-1.99818185e-07  6.88371173e-09]
y  [0.81531903 1.87670627]
tolerância  1.9993672167898356e-07


In [ ]:
fx

4.890023251698579e-17

In [ ]:
m

array([0.81531903, 1.87670627])

In [ ]:
table

,x,y,df(x),Tolerância
0,"[0.8153231343749998, 1.8767061328125]","[0.8153231343749998, 1.8767061328125]","[-0.0016762704299412812, 5.7686012348723514e-05]",1.677263e-03
1,"[0.8153190331255071, 1.8767062739500664]","[0.8153190331255071, 1.8767062739500664]","[-1.9981818532995963e-07, 6.883711729963967e-09]",1.999367e-07


Para este caso, levou apenas duas iterações para a solução com uma precisão na ordem de $10^{-10}$.

**4** Resolva:

Minimizar $(x_1 - 3)^4 + (x_1 - 3x_2)^2$

In [ ]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
   
c = [x1,x2]
x = [1.2,0.5]

fo = (x1 - 3)**4 + (x1 - 3*x2)**2
    
eps = 1e-3
nmax = 7
grad = gradiente_simbolico(fo,c)
p  = Parametros(fo,grad,c,x,eps)
m,fx,table = gradiente_conjugado(p)

d  [1.7042969  5.24183484]
y  [2.16973828 0.42705078]
tolerância  5.5119379955108645
d  [ 1.79358466 -0.00917948]
y  [2.28873948 0.7930578 ]
tolerância  1.7936081541047468
d  [0.69906197 1.05977289]
y  [2.55234738 0.79170867]
tolerância  1.2695693013224167
d  [ 0.37791744 -0.07459683]
y  [2.611399  0.8812305]
tolerância  0.3852093975697302
d  [0.31875778 0.23940123]
y  [2.66214475 0.87121384]
tolerância  0.39864704977634663
d  [ 0.36153443 -0.03313435]
y  [2.72673678 0.91972532]
tolerância  0.36304962022483533
d  [0.09575264 0.10955515]
y  [2.76751533 0.915988  ]
tolerância  0.14550222990159853
d  [ 0.10503349 -0.00178076]
y  [2.77915713 0.92930793]
tolerância  0.10504858105902418
d  [0.13165669 0.08714977]
y  [2.80167163 0.92892622]
tolerância  0.15788782744648877
d  [ 0.13372205 -0.00123914]
y  [2.82732154 0.9459051 ]
tolerância  0.1337277906065852
d  [0.04578217 0.04311623]
y  [2.84449385 0.94574598]
tolerância  0.06288892305344278
d  [ 0.04246655 -0.00128904]
y  [2.85046253 0.95136

In [ ]:
table

,x,y,df(x),Tolerância
0,"[2.16973828125, 0.42705078124999996]","[2.16973828125, 0.42705078124999996]","[1.704296904502253, 5.241834843607758]",5.511938
1,"[2.288739481124913, 0.79305780402144]","[2.288739481124913, 0.79305780402144]","[1.7935846641683018, -0.009179484262031745]",1.793608
2,"[2.5523473834269925, 0.7917086708364441]","[2.5523473834269925, 0.7917086708364441]","[0.6990619670195269, 1.0597728894093674]",1.269569
3,"[2.6113990046644817, 0.8812305018265909]","[2.6113990046644817, 0.8812305018265909]","[0.37791744232189617, -0.07459682811561598]",0.385209
4,"[2.66214475497626, 0.8712138378950507]","[2.66214475497626, 0.8712138378950507]","[0.3187577763614447, 0.23940123288005896]",0.398647
...,...,...,...,...
82,"[2.9769898992508126, 0.9923057098362094]","[2.9769898992508126, 0.9923057098362094]","[0.0019405959882569457, 0.0008875480972670657]",0.002134
83,"[2.9773528134419758, 0.9924716917313525]","[2.9773528134419758, 0.9924716917313525]","[0.0018087030962167217, 0.00037545333361337487]",0.001847
84,"[2.9776733990005337, 0.9925382393681209]","[2.9776733990005337, 0.9925382393681209]","[0.001312369388199949, 0.000639630197658536]",0.001460
85,"[2.9779008849052264, 0.9926491127666408]","[2.9779008849052264, 0.9926491127666408]","[0.0011127315778519998, 0.00019728717670911363]",0.001130


In [ ]:
m

array([2.97809594, 0.9926837 ])

In [ ]:
fx

2.3220807449378338e-07



---



**Conclusões**

O método do Gradiente Conjugado obtém uma precisão muito alta e consegue a convergência, mesmo lenta, quando se trabalha com uma maior número de variáveis. Pode acontecer de chegar a oscilar muito em torno da resposta e não convergir quando os valores são muito distantes da solução ou a precisão requerida não for muito grande. 

Foi observado que para cinco variáveis, quando se tem uma precisão baixa requerida, o método não convergia. A derivada apresentava um valor bastante elevado, o que quer dizer que a solução dava saltos bem grandes na região de busca e se distanciava do ponto de mínimo.

O método levou mais de 1h40min para o problema com 10 variáveis e ainda não havia chegado ao valor de tolerância razoável. O programa foi encerrado ante da convergência, porém notou-se que é possível solucinar.

Pelas conclusões obtidas para 5 e 10 variáveis, torna-se inviável executar o problema para 50 variáveis dado o tempo para a resolver os problemas.

Em geral, o método do Gradinte Conjugado leva muitas mais iterações até a solução, porém para poucas variáveis e iniciando-se de pontos aleátórios,consegue convergir, diferentemente do método de Newton. Em relação ao método de Davidon-Fletcher-Powell, foi observado que o método aqui desenvolvido é menos eficiente em relação ao tempo e à quantidade de iterações. Isso pode estar relacionado à forma como o método foi implementado, tanto no cálculo das variáveis quanto na manipulação do vetores.